# Problem 1 

## (a)

\begin{align*}
& \text{Known a forward contract paying $S_{T_2} − F_t$ at time $T_2$ has time-t value 0: }\\
& \Rightarrow S_t - F_t e^{-r(T_2-t)} = 0 \\ 
& \Rightarrow F_t = S_t  e^{r(T_2-t)}\\

& \text{Constructing the portfolio: }\\
& V_t = (S_t - K e^{-r(T_2-t)}) - (S_t - F_t e^{-r(T_2-t)}) \\
& V_t = S_t - K e^{-r(T_2-t)} - S_t + F_t e^{-r(T_2-t)} = (F_t - K) e^{-r(T_2-t)}\\
& \text{Then, } f_t = (F_t - K) e^{-r(T_2-t)}



\end{align*}

## (b)

The original context: If, say, $F_t > S_te^{r(T_2−t)}$, then arbitrage would exist: at time t, borrow $S_t$ dollars, buy the stock, and short the forward (with delivery price $F_t$ and time-t value 0). At time $T_2$, deliver the stock, and receive $F_t$, which is more than enough to cover your accumulated debt of $S_te^{r(T_2−t)}$ dollars.

The key problem is here that $S_t$ represents the time-t price of crude barrel for time-t delivery. We cannnot simply buy the curde oil at time t, and then return it at $T_2$. We need to consider the storage fee, transportation fee and the default risk of commodity itself for crude oil. Therefore, the acummulated debt cannot be simply estimated as before. The accumultaed debt should equal to $S_te^{r(T_2−t)} + storage fee + transportation fee$. Then, at this point, we are not sure whether $F_t> S_te^{r(T_2−t)} + storage fee + transportation fee $

## (c)

\begin{align*}
& S_t = exp(X_t) \Rightarrow log(S_t) = X_t \\
& d log(S_t) = d X_t = \kappa(\alpha − X_t)dt + \sigma dW_t = \kappa(\alpha − log S_t)dt + \sigma dW_t\\
& \Rightarrow X_t = X_{t-1} +  \kappa(\alpha − X_{t-1}) \times \Delta t + \sigma W_{t-1} \\
& F_t = E_t(S_{T_2}) = \exp \left[ e^{-\kappa (T_2-t)} \log S_t + (1 - e^{-\kappa (T_2-t)}) \alpha + \frac{\sigma^2}{4\kappa} (1 - e^{-2\kappa (T_2-t)}) \right]
\end{align*}

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve
import pandas as pd

In [2]:


class XOU:

    def __init__(self, kappa, alpha, sigma, S0, r):

        self.kappa = kappa
        self.alpha = alpha
        self.sigma = sigma
        self.S0 = S0
        self.r = r

In [3]:
hw5dynamics=XOU(kappa = 0.472, alpha = 4.4, sigma = 0.368, S0 = 106.9, r = 0.05)

In [4]:
class CallOnForwardPrice:

    def __init__(self, K1, T1, T2):

        self.K1 = K1
        self.T1 = T1
        self.T2 = T2


In [5]:
hw5contract=CallOnForwardPrice(K1 = 103.2, T1 = 0.5, T2 = 0.75)

In [6]:
class MCengine:

    def __init__(self, N, M, epsilon, seed):

        self.N = N   
        self.M = M  
        self.epsilon = epsilon  
        self.rng = np.random.default_rng(seed=seed) 

    def price_call_XOU(self, contract, dynamics):


        self.rng.normal()
        S0 = dynamics.S0
        r = dynamics.r
        K1 = contract.K1
        T1 = contract.T1
        T2 = contract.T2
        kappa = dynamics.kappa
        alpha = dynamics.alpha
        sigma = dynamics.sigma
        N = self.N
        M = self.M
        epsilon = self.epsilon
        dt = T1/N
        S0_delta = S0 + epsilon

        def path(S0,S0_delta):
            X0 = np.log(S0)
            X0_delta = np.log(S0_delta)

            for _ in range (1, N+1):

                dW = self.rng.normal()
                #X_t = X_{t-1} +  \kappa(\alpha − X_{t-1}) \times \Delta t + \sigma W_{t-1}
                X0 += (kappa*(alpha - X0)*dt + np.sqrt(dt)*sigma*dW)
                X0_delta += (kappa*(alpha - X0_delta)*dt + sigma*np.sqrt(dt)*dW)
            # \exp \left[ e^{-\kappa (T_2-t)} \log S_t + (1 - e^{-\kappa (T_2-t)}) \alpha + \frac{\sigma^2}{4\kappa} (1 - e^{-2\kappa (T_2-t)}) \right]
            Ft =  np.exp(np.exp(-kappa*(T2-T1))*X0 + (1 - np.exp(-kappa*(T2-T1)))*alpha + sigma**2/(4*kappa)*(1 - np.exp(-2*kappa*(T2-T1))))
            Ft_delta =  np.exp(np.exp(-kappa*(T2-T1))*X0_delta + (1 - np.exp(-kappa*(T2-T1)))*alpha + sigma**2/(4*kappa)*(1 - np.exp(-2*kappa*(T2-T1))))

            call_price = np.exp(-r*T1)*np.maximum(Ft-K1, 0)
            call_price_1 = np.exp(-r*T1)*np.maximum(Ft_delta-K1, 0)

            return(call_price, call_price_1)
        
        call_price_list = []
        call_price_1_list = []
        for _ in range (1, M+1):
            call_price, call_price_1 = path(S0,S0_delta)
            call_price_list.append(call_price)
            call_price_1_list.append(call_price_1)
        
        call_price = np.mean(call_price_list)
        call_price_1 = np.mean(call_price_1_list)
        standard_error = np.std(call_price_list)/np.sqrt(M)
        call_delta = (call_price_1 - call_price)/epsilon
        print(np.exp(np.exp(-kappa*(T2))*np.log(S0) + (1 - np.exp(-kappa*(T2)))*alpha + sigma**2/(4*kappa)*(1 - np.exp(-2*kappa*(T2)))))
        return(call_price, standard_error, call_delta)


In [7]:
hw5MC = MCengine(N=100, M=100000, epsilon=0.01, seed=0)


In [8]:
(call_price, standard_error, call_delta) = hw5MC.price_call_XOU(hw5contract,hw5dynamics)

102.23035226277118


In [9]:
print(call_price, standard_error, call_delta)

7.734508897684603 0.042030256365663675 0.33997421657758764


Standard Error = 0.0420; Call price = 7.7345.

## (d)

Call price delta = 0.3400

## (e)

\begin{align*}

& \text{From question a, we can get: }\\
&f_t = (F_t - K) e^{-r(T_2-t)} \\
& F_t = \exp \left[ e^{-\kappa (T_2-t)} X_t + (1 - e^{-\kappa (T_2-t)}) \alpha + \frac{\sigma^2}{4\kappa} (1 - e^{-2\kappa (T_2-t)}) \right]\\
& \Rightarrow \frac{\partial f_0}{\partial S} = \frac{\partial f_0}{\partial F_t} \frac{\partial F_t}{\partial X_t} \frac{\partial X_t}{\partial S_t}\\

& \Rightarrow \frac{\partial f_0}{\partial S} = e^{-rt}F_0*e^{-\kappa(T_2)}\frac{1}{S_t}\\
& \Rightarrow \frac{\partial f_0}{\partial S} = 102.23035226277118 * e^{-0.472*0.75}\frac{1}{106.9} = 0.67121\\

\end{align*}

## (f)

\begin{align*}
& V_t = -C_t + n \cdot f_t \\
& \frac{\partial V_t}{\partial S_t} = -\frac{\partial C_t}{\partial S_t} + n \cdot \frac{\partial f_t}{\partial S_t}\\
& \Rightarrow - 0.33997421657758764 + n \cdot 0.67121 = 0 \\
& n = 0.33997421657758764/0.67121 \approx 0.5065 
 

\end{align*}

## (g)

\begin{align*}
&V_{T_2} = \theta (F_{T_2} - K) + (5000 - \theta) (S_{T_1} - K)\\
&V_{T_2} = \theta f_{T_2} + (5000 - \theta) C_{T_1}\\
& V_0 = \theta f_0 + (5000 - \theta) C_{T_1} e^{-r(T_2-T_1)}\\
& V_0 = 4000* (102.23035226277118-103.2)*e^{-0.05*0.75} +  1000* 7.734508897684603 e^{-0.05*0.25} \approx 3902.57

\end{align*}

The time-0 value of this contract is approximately 3902.57. 

# Problem 2 

## (a)

The dynamics in 1 is capable of generating a non-constant with resepect to T term-structure of implied volatility. We only need to perform the similar procedure for that random variable. Since it depends on T, it is easy for us to generate it with the $X_t$. But we are unable to generate the implied volatility skew since it is with respect to K, does not match the path dependency created by the dynamics in 1. 

## (b)

\begin{align*}
& S_0 = 100, r = 0.05,p(0.1) = 5.25, p(0.2) = 7.25, p(0.5) = 9.5. \\

& C = N(d_1) S_t - N(d_2)Ke^{-rt}\\
& d_1 = \frac{ln(\frac{S_t}{K} + (r+ \frac{\sigma^2}{2})t)}{\sigma \sqrt{t}} \\
& d_2 = d_1 - \sigma \sqrt{t}

\end{align*}

Utilizing the Black-Scholes Formula, we can get the answer: 

for t $\in$ $[0,0.1]$, $\sigma_{imp}$ = 0.397320

for t $\in$ $(0.1,0.2]$, $\sigma_{imp}$ = 0.362211

for t $\in$ $(0.2,0.5]$, $\sigma_{imp}$ = 0.220871


In [10]:
def black_scholes_call(sigma,T,S =100, K=100, r= 0.05):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

vol_dict = {}
for (p, T) in [(5.25, 0.1), (7.25, 0.2), (9.5, 0.5)]:
    vol_dict[T] = fsolve(lambda x: p - black_scholes_call(x, T), x0=0.3)[0]

In [11]:
vol = pd.DataFrame(list(vol_dict.items()), columns=['T', 'sigma'])
vol['integral'] = vol['sigma'] ** 2 * vol['T']

vol['step_func'] = np.sqrt(
    vol['integral'].diff().fillna(vol['integral']) /
    vol['T'].diff().fillna(vol['T'])
)

vol.index = vol['T']
vol.drop(columns=['T'], inplace=True)
vol

,sigma,integral,step_func
T,,,
0.1,0.397320,0.015786,0.397320
0.2,0.380171,0.028906,0.362211
0.5,0.295097,0.043541,0.220871


## (c)
 
time-0 price is 8.7842. time-0.1 implied volatility is 0.3109.

In [40]:
black_scholes_call(sigma=0.3108971298591372, T=0.4, S =100, K=100, r= 0.05)

8.784201775931614

In [39]:
t = 0.4
np.sqrt((vol.loc[0.2, "integral"] + vol.loc[0.5, "step_func"] ** 2 * (t - 0.2))/t)

0.3108971298591372